# Package 

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
import warnings
warnings.simplefilter('ignore')

import pickle
import gc
import re
import polars as pl
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm
import polars as pl
from utils import *
from src.eval import model_eval
from src.config import raw_data_session_id_dir, candidate_dir, model_for_eval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# df_prod = pd.read_csv('data/products_train.csv')
# df_prod

# Config 

In [4]:
# ! ls ../model_training/

In [5]:
debug = False


# task = 'task2'

# model_dir = f'../model_training/{model_version}/'

# target locales: locales needed for task1
# target_locals = ['ES', 'FR', 'IT']


# model_version = 'popular_v1'

topn = 100
if debug:
    n_rows = 1000
else:
    n_rows = None
# debug_session_num = 100
# train_data_dir = '.'
# test_data_dir = '.'

# if model_for_eval:
# model_file = os.path.join(model_dir, f'nic_{model_for_eval}_for_eval.parquet')
# submit_file = os.path.join('../data/sub_files/', f'submission_{task}_nic_{model_for_eval}_for_eval.parquet')

In [6]:
# candidate_file.format(
#     task='task1'
#     , data_type='train'
#     , model_version=model_version
#     , model_for_eval=model_for_eval
#     , topn=topn
# )

# Data 

In [7]:
train_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))
train2_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train2.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
# task1_test_pl_phase1 = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1_phase1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))

# task2_test_pl_phase1 = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task2_phase1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))

# task3_test_pl_phase1 = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3_phase1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))


task1_test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))

task2_test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task2.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))

task3_test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))

product_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))


In [9]:
# product_pl.head().collect()

# Combine data

In [9]:
cols_to_keep = ['session_id', 'prev_items', 'locale']
session_pl = pl.concat([
    train_pl.select(cols_to_keep),
    train2_pl.select(cols_to_keep),
    eval_pl.select(cols_to_keep), 
    task1_test_pl.select(cols_to_keep),
    task2_test_pl.select(cols_to_keep),
    task3_test_pl.select(cols_to_keep),
    # task1_test_pl_phase1.select(cols_to_keep),
    # task2_test_pl_phase1.select(cols_to_keep),
    # task3_test_pl_phase1.select(cols_to_keep)

],    
    how='vertical')
        

# Session features 

In [12]:
session_len = session_pl.select('session_id').collect().shape[0]
session_len

4014333

In [13]:
original_session_pl = session_pl

In [14]:
original_session_pl.select('session_id').collect().shape

(4014333, 1)

In [15]:
original_session_pl.select('session_id').collect().unique().shape

(4014333, 1)

In [16]:
original_session_pl.schema

{'session_id': Int64, 'prev_items': Unknown, 'locale': Utf8}

In [17]:
original_session_pl.head().collect()

session_id,prev_items,locale
i64,list[str],str
6,"[""B09SMK3R8H"", ""B01N4ND0F9""]","""DE"""
7,"[""B09B2W5S9R"", ""B09B2YFY6M"", ""B09B2WGPRB""]","""DE"""
9,"[""3649625660"", ""B07N3SNQW5"", … ""3848520974""]","""DE"""
11,"[""B0927GXJPB"", ""B003AO3LS6""]","""DE"""
16,"[""B01IKAEFJS"", ""B09LS7L75M"", … ""B08D1XKQLP""]","""DE"""


In [18]:
product_cols = ['id', 'locale', 'price', 'brand', 'color', 'size']
session_pl = (
    session_pl.explode('prev_items')
        .join(product_pl.select(product_cols), 
              left_on=['prev_items', 'locale'], 
             right_on=['id', 'locale']
             )
        .groupby(['session_id', 'locale'])
        .agg(
            pl.all()
        )
        .with_columns(
            pl.col('price').arr.mean().alias('mean_price')
            , pl.col('price').arr.get(-1).alias('last_price')
            , pl.col('price').arr.min().alias('min_price')
            , pl.col('price').arr.max().alias('max_price')
        )
)

In [19]:
session_pl = session_pl.collect()

In [20]:
session_pl.shape[0]

4014333

In [21]:
session_len

4014333

In [22]:
assert session_pl.shape[0] == session_len

In [23]:
session_pl.write_parquet('../data/session_item_features/session_features.parquet')

# Item feature 

In [28]:
def get_seq_position(row):
    length = len(row)
    return list(range(1, length+1))
    # return res

In [37]:
item_features = (
    session_pl
        .with_columns(
            pl.col('prev_items').apply(lambda x: get_cnt(x)).alias('seq_position')
        )
        .explode(['prev_items', 'seq_position'])
        .with_columns(
            pl.lit(1).truediv(pl.col('seq_position')).alias('seq_pos_reverse_weight')
        ).groupby('prev_items')
        .agg(
            pl.count().alias('item_cnt')
            ,pl.col('seq_pos_reverse_weight').sum().alias('pos_weighted_cnt')
            ,pl.col('seq_position').mean().alias('avg_pos')
            ,pl.col('seq_position').median().alias('median_pos')
        )
)

In [39]:
item_features.collect().write_parquet('../data/session_item_features/item_features.parquet')

In [35]:
# (
#     session_pl
#         .with_columns(
#             pl.col('prev_items').apply(lambda x: get_cnt(x)).alias('seq_position')
#         )
#         .explode(['prev_items', 'seq_position'])
#         .with_columns(
#             pl.lit(1).truediv(pl.col('seq_position')).alias('seq_pos_reverse_weight')
#         ).groupby('prev_items')
#         .agg(
#             pl.count().alias('item_cnt')
#             ,pl.col('seq_pos_reverse_weight').sum().alias('pos_weighted_cnt')
#             ,pl.col('seq_position').mean().alias('avg_pos')
#             ,pl.col('seq_position').median().alias('median_pos')
#         )
# ).head().collect()

prev_items,item_cnt,pos_weighted_cnt,avg_pos,median_pos
str,u32,f64,f64,f64
"""B08DJ6FMH8""",2,0.75,3.0,3.0
"""B09KV5WKQM""",1,0.5,2.0,2.0
"""B0070VY3BE""",5,3.833333,1.6,1.0
"""B0767BT2DK""",4,3.0,1.5,1.5
"""B08LZCPM4L""",3,1.583333,2.666667,3.0


In [26]:
# grades = session_pl.head().collect()

In [27]:
# grades

In [25]:
grades.with_columns(
        pl.col('prev_items').apply(lambda x: get_cnt(x)).alias('seq_position')
    )

session_id,prev_items,locale,seq_position
i64,list[str],str,list[i64]
6,"[""B09SMK3R8H"", ""B01N4ND0F9""]","""DE""","[1, 2]"
7,"[""B09B2W5S9R"", ""B09B2YFY6M"", ""B09B2WGPRB""]","""DE""","[1, 2, 3]"
9,"[""3649625660"", ""B07N3SNQW5"", … ""3848520974""]","""DE""","[1, 2, … 7]"
11,"[""B0927GXJPB"", ""B003AO3LS6""]","""DE""","[1, 2]"
16,"[""B01IKAEFJS"", ""B09LS7L75M"", … ""B08D1XKQLP""]","""DE""","[1, 2, … 4]"


# Top200 for fallback logics

In [11]:
train_data = train_pl.with_columns(
    pl.col('prev_items').arr.concat(pl.col('next_item')) 
)
if not model_for_eval:
    eval_data = eval_pl.with_columns(
    pl.col('prev_items').arr.concat(pl.col('next_item')) 
)
else:
    eval_data = eval_pl
test_data = test_pl
cols_to_keep = ['prev_items', 'locale']
popular_pl = pl.concat([train_data.select(cols_to_keep), eval_data.select(cols_to_keep), test_data.select(cols_to_keep)], how='vertical')
        

In [12]:
# next_item_df[cols].info()

In [13]:
popular_pl.head().collect()

prev_items,locale
list[str],str
"[""B09W9FND7K"", ""B09JSPLN1M"", ""B09M7GY217""]","""DE"""
"[""B076THCGSG"", ""B007MO8IME"", … ""B001B4THSA""]","""DE"""
"[""B0B1LGXWDS"", ""B00AZYORS2"", … ""B0767DTG2Q""]","""DE"""
"[""B0749V8TC7"", ""B0749W93VC"", … ""B0749TX4YS""]","""DE"""
"[""B09SMK3R8H"", ""B01N4ND0F9"", ""B08YNZT93Z""]","""DE"""


In [14]:
topn = 200
locale_popular_pl = (
    popular_pl
        .explode('prev_items')
        .groupby(['locale', 'prev_items'])
        .agg(
            pl.count()
        )
        .with_columns(
            pl.col('count').rank(method='ordinal', descending=True).over('locale').alias('rank')
        )
        .filter(pl.col('rank')<=topn)
        .with_columns(
            pl.col('count').max().over('locale').alias('max_count')
            , pl.col('count').min().over('locale').alias('min_count')
        )
        .with_columns(
            ((pl.col('count')-pl.col('min_count'))/(pl.col('max_count')-pl.col('min_count'))).alias('weight')
        )
        .sort('locale', 'rank')
        .select(
            'locale'
            , 'prev_items'
            , 'weight'
        )
        .groupby('locale')
        .agg(
            pl.col('weight').alias('locale_popular_weight')
            , pl.col('prev_items').alias('locale_popular_rec')
        )
        # .count()#.head(3).collect())
        # .collect()
)

In [15]:
locale_popular_pl.collect()

locale,locale_popular_weight,locale_popular_rec
str,list[f64],list[str]
"""UK""","[1.0, 0.911864, … 0.0]","[""B08CN3G4N9"", ""B07N8QY3YH"", … ""B0765ZQJYJ""]"
"""JP""","[1.0, 0.668449, … 0.0]","[""B0BD5MFPMF"", ""B0BD88WWQ8"", … ""B07PNGSC5S""]"
"""DE""","[1.0, 0.963384, … 0.0]","[""B0BDML9477"", ""B08GWS298V"", … ""B01GGKYLW0""]"


# Model Eval 

In [26]:
def get_next_items(x):
    prev_items = x['prev_items']
    local_rec = x['locale_popular_rec']
    final = [ele for ele in local_rec if ele not in prev_items]
    return final


eval_res = (
    eval_pl
        .join(locale_popular_pl, how='left', on='locale')
        .with_columns(
            pl.col('locale_popular_rec')#.alias('next_item_prediction')
            , pl.struct(["prev_items", "locale_popular_rec"]).apply(
                        lambda x: get_next_items(x)).alias('next_item_prediction')
        )
)

# eval_res.head().collect()

In [27]:
%%time
model_eval(eval_res)

CPU times: user 4min 33s, sys: 1min 21s, total: 5min 54s
Wall time: 1min 59s


total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
326443,0.000145,0.000493,0.00227


In [18]:
# eval_res.head().collect()

In [28]:
# model_eval(eval_res)

# Save Model

In [32]:
model_dir

'../model_training/popular_v1/'

In [31]:
popular_pl.collect().write_parquet(os.path.join(model_dir, 'popular_rec.parquet'))